In [1]:
import syft as sy

/Users/madhavajay/.local/share/virtualenvs/PySyft-lHlz_cKe/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:125: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


In [2]:
# we need this to make a local tensor to send instead of using primitives locally
import torch as local_torch

alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()
alice.root_verify_key = alice_client.verify_key  # inject 📡🔑 as 📍🗝

> Creating 🗃️ (MemoryStore) {}
> Creating 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪>


In [3]:
# get imports from remote client
torch = alice_client.torch

In [4]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.optim.lr_scheduler import StepLR

In [5]:
# send a tensor so we can use the pointer for tensor creation here
# because we cant yet serialize primitives
aone = local_torch.tensor([1], dtype=local_torch.uint8)
print(aone, type(aone))
xone = aone.send(alice_client)

tensor([1], dtype=torch.uint8) <class 'torch.Tensor'>
> Creating ✉️  (SaveObjectAction) <UID:🚰🚧>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing ✉️  (SaveObjectAction) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚰🚧>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating ✉️  (SaveObjectAction) <UID:🚇🙼>


In [6]:
print(alice.store.peek)

{
  > <UID:4581403d-a6a5-4725-97cc-9aed710454e1> <UID:🚁🙎> => 🗂️ (StorableObject) (tensor([1], dtype=torch.uint8) can_read: ['🜣', '🜥'])
}


In [7]:
a = torch.Generator()

call torch.Generator on client <VirtualMachineClient id:alice Client>
> Creating ✉️  (RunFunctionOrConstructorAction) <UID:🚱🚸>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing ✉️  (RunFunctionOrConstructorAction) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚱🚸>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating ✉️  (RunFunctionOrConstructorAction) <UID:🚱🚸>


In [8]:
print(a.id_at_location, type(a))
x = a.get_state()
x.id_at_location

<UID:5e5982d3-3d75-4712-86a9-ece7e297010b> abc.GeneratorPointer
> Creating RunClassMethodAction(torch.Generator.get_state) <UID:🚋🚴>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing RunClassMethodAction(torch.Generator.get_state) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚋🚴>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating RunClassMethodAction(torch.Generator.get_state) <UID:🚋🚴>


<UID:5fe3bb2f-07f4-4c30-bc5b-184a55be54fb>

In [9]:
c = x.get()

> Creating GetObjectAction(<UID:5fe3bb2f-07f4-4c30-bc5b-184a55be54fb>) <UID:🙺🚋>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing GetObjectAction(<UID:5fe3bb2f-07f4-4c30-bc5b-184a55be54fb>) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithReply) <UID:🙺🚋>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating GetObjectAction(<UID:5fe3bb2f-07f4-4c30-bc5b-184a55be54fb>) <UID:🙺🚋>
> Creating ✉️  (GetObjectResponseMessage) <UID:🚇🛢>
> Signing with 🜣
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚇🛢>
> 🍰 alice (VirtualMachine)@<UID:🚱🚪> Signing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) with 🜣
> Creating ✉️  (GetObjectResponseMessage) <UID:🚇🛢>


In [10]:
print(c)
print(type(c))
print(c.id)

tensor([  1, 209, 156,  ...,   0,   0,   0], dtype=torch.uint8)
<class 'torch.Tensor'>
<UID:a1d93b56-8430-45e3-87a1-b562ebf0edf8>


In [11]:
# d = c + 1
# print(type(d))
# print(d.id)

In [12]:
cx = c.send(alice_client)

> Creating ✉️  (SaveObjectAction) <UID:🚵🚳>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing ✉️  (SaveObjectAction) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚵🚳>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating ✉️  (SaveObjectAction) <UID:🚉🚰>


In [13]:
print(alice.store.peek)

{
  > <UID:4581403d-a6a5-4725-97cc-9aed710454e1> <UID:🚁🙎> => 🗂️ (StorableObject) (tensor([1], dtype=torch.uint8) can_read: ['🜣', '🜥'])

  > <UID:5e5982d3-3d75-4712-86a9-ece7e297010b> <UID:🙵🚉> => 🗂️ (StorableObject) (<torch._C.Generator object at 0x1302591e0>)

  > <UID:a1d93b56-8430-45e3-87a1-b562ebf0edf8> <UID:🛩🛟> => 🗂️ (StorableObject) (tensor([  1, 209, 156,  ...,   0,   0,   0], dtype=torch.uint8) can_read: ['🜣', '🜥'])
}


In [14]:
# from MNIST we need to run:
# torch.manual_seed(args.seed)

xseed = torch.manual_seed(xone)
xseed
# print(xseed.id_at_location, type(xseed))

call torch.manual_seed on client <VirtualMachineClient id:alice Client>
> Creating ✉️  (RunFunctionOrConstructorAction) <UID:🚅🚊>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing ✉️  (RunFunctionOrConstructorAction) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚅🚊>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating ✉️  (RunFunctionOrConstructorAction) <UID:🚅🚊>


In [15]:
statex = xseed.get_state()
tensor_state = statex.get()
print(tensor_state)

> Creating RunClassMethodAction(torch.Generator.get_state) <UID:🙽🚳>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing RunClassMethodAction(torch.Generator.get_state) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🙽🚳>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating RunClassMethodAction(torch.Generator.get_state) <UID:🙽🚳>
> Creating GetObjectAction(<UID:929a339d-6e66-4d5e-a249-458b3eb09d58>) <UID:🚌🙸>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing G

In [16]:
xseed.set_state(cx)

> Creating RunClassMethodAction(torch.Generator.set_state) <UID:🙽🙖>
> 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> Signing RunClassMethodAction(torch.Generator.set_state) with 🜥
> Signing with 🜥
> Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🙽🙖>
> Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [🍰] alice Client (VirtualMachineClient)@<UID:🚱🚪> ➡️  💠 [🍰] alice Client (Address)
> Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
> Received ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Processing 📨 ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) @ 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Recipient Found ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply)@<UID:🚱🚪> == 🍰 alice (VirtualMachine)@<UID:🚱🚪>
> Creating RunClassMethodAction(torch.Generator.set_state) <UID:🙽🙖>


In [17]:
print(alice.store.peek)

{
  > <UID:4581403d-a6a5-4725-97cc-9aed710454e1> <UID:🚁🙎> => 🗂️ (StorableObject) (tensor([1], dtype=torch.uint8) can_read: ['🜣', '🜥'])

  > <UID:5e5982d3-3d75-4712-86a9-ece7e297010b> <UID:🙵🚉> => 🗂️ (StorableObject) (<torch._C.Generator object at 0x1302591e0>)

  > <UID:a1d93b56-8430-45e3-87a1-b562ebf0edf8> <UID:🛩🛟> => 🗂️ (StorableObject) (tensor([  1, 209, 156,  ...,   0,   0,   0], dtype=torch.uint8) can_read: ['🜣', '🜥'])

  > <UID:454b61f9-8e65-4aea-88a2-2aa675e2d116> <UID:🙾🚅> => 🗂️ (StorableObject) (<torch._C.Generator object at 0x11fd81c30> can_read: ['🜣', '🜥'])

  > <UID:3d52dec8-4b94-445c-8cd7-c245ee6f57b9> <UID:🚉🛨> => 🗂️ (StorableObject) (<torch._C.Generator object at 0x11fd81c30> can_read: ['🜥', '🜣'])
}


In [19]:
torch.cuda.is_available()

AttributeError: 'Module' object has no attribute 'cuda'

In [ ]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, 1)
#         self.conv2 = nn.Conv2d(32, 64, 3, 1)
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.dropout2 = nn.Dropout2d(0.5)
#         self.fc1 = nn.Linear(9216, 128)
#         self.fc2 = nn.Linear(128, 10)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.dropout1(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         output = F.log_softmax(x, dim=1)
#         return output

In [ ]:
# def train(args, model, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, target)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % args.log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
#             if args.dry_run:
#                 break

In [ ]:
# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))

In [ ]:
# def main():
#     # Training settings
#     parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
#     parser.add_argument('--batch-size', type=int, default=64, metavar='N',
#                         help='input batch size for training (default: 64)')
#     parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                         help='input batch size for testing (default: 1000)')
#     parser.add_argument('--epochs', type=int, default=14, metavar='N',
#                         help='number of epochs to train (default: 14)')
#     parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
#                         help='learning rate (default: 1.0)')
#     parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
#                         help='Learning rate step gamma (default: 0.7)')
#     parser.add_argument('--no-cuda', action='store_true', default=False,
#                         help='disables CUDA training')
#     parser.add_argument('--dry-run', action='store_true', default=False,
#                         help='quickly check a single pass')
#     parser.add_argument('--seed', type=int, default=1, metavar='S',
#                         help='random seed (default: 1)')
#     parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                         help='how many batches to wait before logging training status')
#     parser.add_argument('--save-model', action='store_true', default=False,
#                         help='For Saving the current Model')
#     args = parser.parse_args()
#     use_cuda = not args.no_cuda and torch.cuda.is_available()

#     torch.manual_seed(args.seed)

#     device = torch.device("cuda" if use_cuda else "cpu")

#     kwargs = {'batch_size': args.batch_size}
#     if use_cuda:
#         kwargs.update({'num_workers': 1,
#                        'pin_memory': True,
#                        'shuffle': True},
#                      )

#     transform=transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
#         ])
#     dataset1 = datasets.MNIST('../data', train=True, download=True,
#                        transform=transform)
#     dataset2 = datasets.MNIST('../data', train=False,
#                        transform=transform)
#     train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
#     test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

#     model = Net().to(device)
#     optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

#     scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
#     for epoch in range(1, args.epochs + 1):
#         train(args, model, device, train_loader, optimizer, epoch)
#         test(model, device, test_loader)
#         scheduler.step()

#     if args.save_model:
#         torch.save(model.state_dict(), "mnist_cnn.pt")

In [ ]:
# if __name__ == '__main__':
#     main()